In [1]:
import scanpy as sc
import numpy as np
import scipy
import sys
from tqdm import tqdm

This notebook we generate a normalized count matrix from the integrated seurat file masking all other genes as zeroes. Note this requires >300gb memory.

In [2]:
# Load integrated data
adata = sc.read_h5ad("./objects/integrated_seurat.h5ad")
adata

/home/zatzmanm/work/miniconda3/envs/infercnvpy/lib/python3.10/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 929690 × 3000
    obs: 'percent.mt', 'percent.rb', 'doublet', 'sample', 'batch', 'S.Score', 'G2M.Score', 'Phase', 'CC.Diff', 'cell_type', 'nCount_RNA', 'nFeature_RNA', 'doublet_score', 'cell_type_super', 'patient_id', 'seurat_clusters', 'cell_id'
    var: 'gene.ids', 'feature.types', 'highly.variable', 'mvp.mean', 'mvp.dispersion', 'mvp.dispersion.scaled', 'highly.variable.nbatches', 'highly.variable.intersection', 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap50'
    varm: 'PCs'
    obsp: 'distances'

In [3]:
# 3000 genes in X
adata.X

array([[-0.18643191, -0.38347755,  0.88487009, ..., -0.47285926,
        -0.48092199, -0.14139366],
       [-0.18643191, -0.38347755, -0.32239258, ..., -0.47285926,
        -0.48092199, -0.14139366],
       [-0.18643191, -0.38347755, -0.52133864, ..., -0.47285926,
        -0.48092199, -0.14139366],
       ...,
       [-0.18643191, -0.38347755,  2.28687729, ...,  2.31006705,
         2.08383455, -0.14139366],
       [-0.18643191, -0.38347755, -0.52186927, ...,  0.73835389,
         0.97856159, -0.14139366],
       [-0.18643191, -0.38347755,  1.64683753, ...,  2.18301091,
         2.50387491, -0.14139366]])

In [4]:
raw_counts = sc.read_h5ad(filename="/work/shah/isabl_data_lake/analyses/84/78/8478/RNASCP/RNASCP/BUILD_MATRIX/fork0/files/matrix.h5")

raw_counts

AnnData object with n_obs × n_vars = 1270951 × 32175
    var: 'gene_ids', 'feature_types'

In [5]:
# subset raw counts for cells in adata
adata.obs.head()

,percent.mt,percent.rb,doublet,sample,batch,S.Score,G2M.Score,Phase,CC.Diff,cell_type,nCount_RNA,nFeature_RNA,doublet_score,cell_type_super,patient_id,seurat_clusters,cell_id
SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACCCACACTGGAAG,12.332594,28.637371,0,237,35,-0.100605,0.296480,1,-0.397085,Ovarian.cancer.cell,32929,5833,0.147368,Ovarian.cancer.super,35,3,SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACCCAC...
SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACGAAGTTCGAGCC,22.090894,17.706761,0,237,35,-0.281512,0.916535,1,-1.198046,Ovarian.cancer.cell,31135,5788,0.223022,Ovarian.cancer.super,35,3,SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACGAAG...
SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACGAAGTTTCCATT,19.802657,20.105933,0,237,35,0.404795,-0.044553,2,0.449348,Ovarian.cancer.cell,23411,4881,0.030928,Ovarian.cancer.super,35,3,SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACGAAG...
SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACGCTCACAACGAG,16.439193,21.430026,0,237,35,-0.031933,0.058618,1,-0.090550,Ovarian.cancer.cell,9818,3102,0.109677,Ovarian.cancer.super,35,3,SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAACGCTC...
SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAAGAACAGAGCATAT,21.249505,16.229981,0,237,35,1.320709,0.009225,2,1.311484,Ovarian.cancer.cell,17671,4238,0.109677,Ovarian.cancer.super,35,18,SPECTRUM-OV-107_S1_CD45N_RIGHT_ADNEXA_AAAGAACA...


In [6]:
raw_counts = raw_counts[adata.obs.cell_id]
raw_counts

View of AnnData object with n_obs × n_vars = 929690 × 32175
    var: 'gene_ids', 'feature_types'

In [7]:
all(raw_counts.obs.index.tolist() == adata.obs['cell_id'])

True

In [8]:
# Determine which genes are filtered in the normalized matrix
kept_ids = set(adata.var['gene.ids'])
all_ids = set(raw_counts.var['gene_ids'])

# These need to be flagged in 'var' as 'feature_is_filtered' and have counts zeroed
filtered_ids = all_ids.difference(kept_ids)
len(filtered_ids)

29175

In [12]:
# Index on gene ids which are unique
raw_counts.var['gene_name'] = raw_counts.var.index
raw_counts.var.set_index("gene_ids", inplace=True)
raw_counts.var

,feature_types,gene_name
gene_ids,,
ENSG00000243485,Gene Expression,MIR1302-2HG
ENSG00000186092,Gene Expression,OR4F5
ENSG00000238009,Gene Expression,AL627309.1
ENSG00000239945,Gene Expression,AL627309.3
ENSG00000241599,Gene Expression,AL627309.4
...,...,...
ENSG00000276345,Gene Expression,AC004556.1
ENSG00000277856,Gene Expression,AC233755.2
ENSG00000275063,Gene Expression,AC233755.1


In [14]:
# Create sparse matrix of zeros equal in size
# LIL format allows better slicing
raw_counts.X = scipy.sparse.lil_array(raw_counts.X.shape)

In [15]:
# Loop over and replace gene values for the cells
for i, gene in enumerate(tqdm(adata.var['gene.ids'])):
    gene_idx = np.where(raw_counts.var.index == gene)[0]    
    raw_counts.X[:,gene_idx] = adata.X[:,i]

100%|██████████| 3000/3000 [16:31<00:00,  3.03it/s]


In [22]:
# Convert to sparse format and save out (in case memory blows up and need to reload)
raw_counts.X = scipy.sparse.csr_matrix(raw_counts.X)

In [23]:
raw_counts.X

<929690x32175 sparse matrix of type '<class 'numpy.float64'>'
	with 2789070000 stored elements in Compressed Sparse Row format>

In [24]:
raw_counts.write_h5ad(filename="objects/00_zeroes_plus_normalized_csr.h5ad", compression = "gzip")

In [32]:
# Reload and initialize raw counts matrix...
raw_countmat = sc.read_h5ad(filename="/work/shah/isabl_data_lake/analyses/84/78/8478/RNASCP/RNASCP/BUILD_MATRIX/fork0/files/matrix.h5")

In [33]:
raw_countmat = raw_countmat[adata.obs.cell_id]

In [37]:
raw_countmat.X

<929690x32175 sparse matrix of type '<class 'numpy.float32'>'
	with 2181355725 stored elements in Compressed Sparse Row format>

In [40]:
# Initialize raw counts matrix in raw
raw_counts.raw = raw_countmat
raw_counts.raw.X

<929690x32175 sparse matrix of type '<class 'numpy.float32'>'
	with 2181355725 stored elements in Compressed Sparse Row format>

In [ ]:
raw_counts.X = scipy.sparse.csr_matrix(raw_counts.X, dtype="float32")

In [41]:
raw_counts.write_h5ad(filename="objects/01_raw_plus_normalized.h5ad", compression = "gzip")